In [11]:
import numpy as np
import scipy
import utils
from time import time
from utils import HouseHolder, QR, SVD

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [47]:
A = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 0],
              [2, 5, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]], dtype=np.float64).T


# A = np.array([[1, 0, 1],
#               [2, 5**.5, 0],
#               [0, 0, 1],
#               [0, 0, 1]])
B, Qt, P = SVD.svd_phaseI(A)
print(B)


[[ 2.  0.  0.  0.  0.  0.  0.]
 [-5.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


In [48]:
print(Qt @ A @ P)

[[ 2.  0.  0.  0.  0.  0.  0.]
 [-5.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


## Phase-II

In [21]:
m = n = 500
A = np.random.rand(m,n)

**Test Fast Multiplication for A@B, where B is upper bidiagonal matrix.**

In [5]:
B, _, _ = SVD.svd_phaseI(A)

In [12]:
numpy_mul_begin = time()
for i in range(2000):
    A@B
numpy_mul_end = time()
numpy_mul_end - numpy_mul_begin

58.8654887676239

In [14]:
fastMul_begin = time()
for i in range(2000):
    SVD.fastMult_upper_bidiagonal(A, B)
fastMul_end = time()
fastMul_end - fastMul_begin

46.62187671661377

**Test SVD**

Choose the parameter phaseII as 'A', 'B1', 'B2' to test different implementations of phase II

In [45]:
m = n = 500
A = np.random.rand(m,n)

In [46]:
A[490:, :] = 0

In [47]:
U, S, Vt = SVD.svd(A, phaseII='A')
U, S, Vt

phaseI: 2.350073s
phaseII: 4.665369s


(array([[ 0.0673, -0.0535, -0.0159, ...,  0.0009, -0.0439, -0.05  ],
        [ 0.0811,  0.024 ,  0.0158, ...,  0.051 , -0.0597, -0.0292],
        [ 0.0098, -0.0698, -0.0312, ..., -0.0939,  0.0077, -0.0221],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]]),
 array([2.4754e+02, 1.2562e+01, 1.2442e+01, 1.2387e+01, 1.2367e+01,
        1.2308e+01, 1.2206e+01, 1.2163e+01, 1.2141e+01, 1.2136e+01,
        1.2076e+01, 1.1924e+01, 1.1865e+01, 1.1808e+01, 1.1752e+01,
        1.1736e+01, 1.1691e+01, 1.1613e+01, 1.1576e+01, 1.1569e+01,
        1.1490e+01, 1.1443e+01, 1.1435e+01, 1.1386e+01, 1.1331e+01,
        1.1273e+01, 1.1235e+01, 1.1165e+01, 1.1147e+01, 1.1110e+01,
        1.1056e+01, 1.1023e+01, 1.0976e+01, 1.0951e+01, 1.0919e+01,
        1.0864e+01, 1.0794e+01, 1.0786e+01, 1.0739e+01, 1.0730e+01,
        1.0691e+01, 1.0635e

**Accuracy Test:**

In [50]:
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")

Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
1.7999999999999998 %


**Scipy SVD**

In [38]:
U, S, Vt  = scipy.linalg.svd(A, full_matrices=False)
# print(np.abs(U@np.diag(S)@Vt - A))
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")
# U, S, Vt

Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
100.0 %
